In [1]:
%matplotlib inline
%pylab inline
%load_ext autotime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import seaborn as sns
import os
import glob
import geopandas as gpd
import geopy
import plotly_express as px
import tqdm
import pickle

from graphics import Point
from functools import reduce
from datetime import date
from datetime import datetime
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm.notebook import tqdm_notebook


#from dtale.views import startup
#import dtale
#import pyplot as plt
# os and glob help concat multiple csv files into one

Populating the interactive namespace from numpy and matplotlib
time: 2.42 s (started: 2021-12-20 00:59:34 -06:00)


In [2]:
redfinandzillow = pd.read_csv('../data/RedfinandZillowdata.csv')

time: 32 ms (started: 2021-12-20 00:59:36 -06:00)


In [3]:
finalairbnbdata = pd.read_csv('../data/FinalAirBnBData.csv')

time: 219 ms (started: 2021-12-20 00:59:38 -06:00)


In [4]:
redfinandzillow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10665 entries, 0 to 10664
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   SOLD DATE   600 non-null    object 
 1   Street      2002 non-null   object 
 2   CITY        700 non-null    object 
 3   Zipcode     700 non-null    float64
 4   PRICE       700 non-null    float64
 5   Bedroom_x   569 non-null    float64
 6   Bathroom_x  573 non-null    float64
 7   sqft_x      581 non-null    float64
 8   Latitude    700 non-null    float64
 9   Longitude   700 non-null    float64
 10  Address     1309 non-null   object 
 11  Price       1309 non-null   object 
 12  Bedroom_y   1309 non-null   object 
 13  Bathroom_y  1309 non-null   object 
 14  sqft_y      1309 non-null   object 
 15  Sold        1309 non-null   object 
 16  latitude    8653 non-null   float64
 17  longitude   8653 non-null   float64
 18  City        1309 non-null   object 
dtypes: float64(9), object(10)

In [6]:
redfinandzillow = redfinandzillow.drop_duplicates(subset=['Street'])

time: 16 ms (started: 2021-12-20 01:00:23 -06:00)


In [5]:
finalairbnbdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56798 entries, 0 to 56797
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             56798 non-null  int64  
 1   AirBnB         53592 non-null  object 
 2   HostID         53592 non-null  float64
 3   Host           52317 non-null  object 
 4   Latitude       53592 non-null  float64
 5   Longitude      53592 non-null  float64
 6   Room_Type      53592 non-null  object 
 7   Price          53592 non-null  float64
 8   Min_Nights     53592 non-null  float64
 9   Reviews        53592 non-null  float64
 10  Last_Review    46461 non-null  object 
 11  Rev_per_month  46461 non-null  float64
 12  HostListings   53592 non-null  float64
 13  Availability   53592 non-null  float64
 14  Address        53592 non-null  object 
 15  Street         53592 non-null  object 
 16  City           53592 non-null  object 
 17  State          53592 non-null  object 
 18  Countr

In [7]:
finalairbnbdata = finalairbnbdata.drop_duplicates(subset=['Street'])

time: 16 ms (started: 2021-12-20 01:00:48 -06:00)


In [ ]:
#if i didnt drop duplicates, i would have gotte approx 22.7m rows when merging these two df's

In [8]:
bought_in_2021sample = pd.merge(finalairbnbdata, redfinandzillow, on = 'Street', how='inner')

time: 15 ms (started: 2021-12-20 01:01:18 -06:00)


In [11]:
bought_in_2021sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 0 to 26
Data columns (total 37 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             27 non-null     int64  
 1   AirBnB         26 non-null     object 
 2   HostID         26 non-null     float64
 3   Host           25 non-null     object 
 4   Latitude_x     26 non-null     float64
 5   Longitude_x    26 non-null     float64
 6   Room_Type      26 non-null     object 
 7   Price_x        26 non-null     float64
 8   Min_Nights     26 non-null     float64
 9   Reviews        26 non-null     float64
 10  Last_Review    22 non-null     object 
 11  Rev_per_month  22 non-null     float64
 12  HostListings   26 non-null     float64
 13  Availability   26 non-null     float64
 14  Address_x      26 non-null     object 
 15  Street         26 non-null     object 
 16  City_x         26 non-null     object 
 17  State          26 non-null     object 
 18  Country     

In [13]:
bought_in_2021sample['Street']

0       2123 Ashwood Ave
1      6350 Columbia Ave
2          515 N 17th St
3      2245 Berrywood Rd
4      1433 Electric Ave
5      4910 Delaware Ave
6      509 Cottonwood Dr
7       1026 Fairwin Ave
8           810 Glen Ave
9        1104 18th Ave S
10        1917 Cephas St
11         50 Waters Ave
12         6521 Lampe Ct
13       2417 Carter Ave
14      1221 Pennock Ave
15    703 Tulip Grove Rd
16         162 3rd Ave N
17    923 W Eastland Ave
18     1043 E Trinity Ln
19     602 Inverness Ave
20        312 Pioneer Ln
21      1105 Clayton Ave
22       1508 Norvel Ave
23          702 Ries Ave
24        1629 7th Ave N
25     1300 Saunders Ave
26                   NaN
Name: Street, dtype: object

time: 0 ns (started: 2021-12-20 01:03:30 -06:00)


In [14]:
bought_in_2021sample.to_csv("SoldandturnedAirBnB.csv", sep=',', index=False, header=True)

time: 0 ns (started: 2021-12-20 18:14:07 -06:00)
